In [46]:
import numpy as np
import pandas as pd
import os
import sys
import clr

# Change this to suit your PLEXOS version & Location
sys.path += ['C:\\Program Files\\Energy Exemplar\\PLEXOS 9.2 API']
sys.path += ['C:\\Program Files\\Energy Exemplar\\PLEXOS 9.2']

clr.AddReference('PLEXOS_NET.Core')
clr.AddReference('EEUTILITY')
clr.AddReference('EnergyExemplar.PLEXOS.Utility')

from EEUTILITY.Enums import *
from EnergyExemplar.PLEXOS.Utility.Enums import *
from EnergyExemplar.PLEXOS.Utility import *
from PLEXOS_NET.Core import DatabaseCore
from EnergyExemplar import *
import EEUTILITY

# First some simple functions to create, load and save PLEXOS dataset xml files.

In [47]:
def create_target_db(dbFile):
    # creates a new PLEXOS dataset at dbFile filepath
    db = DatabaseCore()
    db.NewEmptyDatabase(dbFile, True)
    db.Connection(dbFile)
    db.Close()
    return

def connect_db(dbFile):
    # point the api to dataset
    db = DatabaseCore()
    db.Connection(dbFile)
    return db

def close_db(db):
    #close and save the dataset
    db.Close()
    print('Database Saved')
    return 

# First let's make some dummy data to create an example

In [52]:
power_plants = ['Solar_NO','Solar_DK','Offshore_Wind_NO',
                'Offshore_Wind_DK','Hydrogen_CCGT_DK',
                'CCGT_DK','Unlimited_Hydro_NO']
cols = ['Capacity (MW)','Type','Fuel Emission Factor (g/kWh)','Heat Rate','Node']

dummy_data = [[500.0,'PV',None,None,'NO'],
              [1200.0,'PV',None,None,'DK'],
              [5000.0,'Offshore Wind',None,None,'NO'],
              [6000.0,'Offshore Wind',None,None,'DK'],
              [3000.0,'Hydrogen',0.0,2.0,'DK'],
              [3000.0,'Fossil Gas',0.2,2.0,'DK'],
              [10000.0,'Hydro',None,None,'NO']]

df_powerplants = pd.DataFrame(index=power_plants,
                              columns=cols,
                              data=dummy_data)

print('Some dummy power plant data:')
display(df_powerplants)


nodes = df_powerplants['Node'].unique()
cols = nodes

df_transmission = pd.DataFrame(index=nodes,columns = nodes,)
df_transmission.loc['NO','DK'] = 7000
df_transmission.loc['DK','NO'] = 7000

print('Some dummy transmission capacity data:')
display(df_transmission)

Some dummy power plant data:


,Capacity (MW),Type,Fuel Emission Factor (g/kWh),Heat Rate,Node
Solar_NO,500.0,PV,NaN,NaN,NO
Solar_DK,1200.0,PV,NaN,NaN,DK
Offshore_Wind_NO,5000.0,Offshore Wind,NaN,NaN,NO
Offshore_Wind_DK,6000.0,Offshore Wind,NaN,NaN,DK
Hydrogen_CCGT_DK,3000.0,Hydrogen,0.0,2.0,DK
CCGT_DK,3000.0,Fossil Gas,0.2,2.0,DK
Unlimited_Hydro_NO,10000.0,Hydro,NaN,NaN,NO


Some dummy transmission capacity data:


,NO,DK
NO,NaN,7000
DK,7000,NaN


# Now let's start creating objects - some basic oven-ready functions

In [58]:
def create_region(db,region):
    # create a region with demand specified in a 'demand' datafile object
    db.AddObject(region, EEUTILITY.Enums.ClassEnum.Region, True)
    # we now have to retrieve the membership id between the new object and the parent system object
    membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemRegions,"System",region)
    # we also need the property id aka the property 'enum' in Energy Exemplar terminology
    demand_enum = db.PropertyName2EnumId("System", "Region", "Regions", "Load")
    # we can then add a property to the new object using the membership id
    db.AddProperty(membership_id, demand_enum, 1, 0, None, None, None, 'demand', None, None, None, PeriodEnum.Interval)
    return db


def create_node(db,region):
    # create a node and assign it to the corresponding region with the same name
    db.AddObject(region, EEUTILITY.Enums.ClassEnum.Node, True)
    # assign the node to the region using the collection enum
    # api structure (e.g. using colleciton enum etc.) is initially confusing but becomes very repetetive
    # you don't need to understand it all straight away. Trial and error helps.
    db.AddMembership(CollectionEnum.NodeRegion,region,region)    
    return db


def create_line(db,node_from,node_to,capacity):
    # create a transmission line from node_from to node_to
    
    # first create the transmission line object
    db.AddObject(node_from + '-'+node_to,ClassEnum.Line, True)
    # then add a membership with the exporting node
    db.AddMembership(CollectionEnum.LineNodeFrom,node_from + '-'+node_to,node_from)
    # ...and with the importing node too
    db.AddMembership(CollectionEnum.LineNodeTo,node_from + '-'+node_to,node_to)
    # again get the membership id of the object with the system
    membership_id = db.GetMembershipID(CollectionEnum.SystemLines,"System",node_from + '-'+node_to)
    # and the proprty id for any properties we want to define
    capacity_enum = db.PropertyName2EnumId("System", "Line", "Lines", "Max Flow")
    # add the property
    db.AddProperty(membership_id, capacity_enum, 1, capacity,
                   None, None, None, None, None, None, None, PeriodEnum.Interval)
    return db


def create_lines(db,df_transmission):
    #create all transmission lines as specified by the transmission dataframe
    for i in range(0,len(df_transmission.columns)):
        node_from = df_transmission.index[i]
        for j in range(i+1,len(df_transmission.columns)):
            node_to = df_transmission.index[j]
            db = create_line(db,node_from,node_to,df_transmission.loc[node_from,node_to])
    return db



def create_datafiles(db):
    #create datafile object
    # fuel prices
    db.AddObject('prices', EEUTILITY.Enums.ClassEnum.DataFile, True)
    membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemDataFiles,"System",'prices')
    path_enum = db.PropertyName2EnumId("System", "Data File", "Data Files", "Filename")
    db.AddProperty(membership_id, path_enum, 1, 0, None, None, None, 'Inputs\Prices.csv',
                   None, None, None, PeriodEnum.Interval)
    
    # onshore wind
    db.AddObject('ratings', EEUTILITY.Enums.ClassEnum.DataFile, True)
    membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemDataFiles,"System",'ratings')
    db.AddProperty(membership_id, path_enum, 1, 0, None, None, None, 'Inputs\renewableprofiles.csv',
                   None, None, None, PeriodEnum.Interval)
        
    # demand
    db.AddObject('demand', EEUTILITY.Enums.ClassEnum.DataFile, True)
    membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemDataFiles,"System",'demand')
    db.AddProperty(membership_id, path_enum, 1, 0, None, None, None, 'Inputs\Load.csv', None, None, None, PeriodEnum.Interval)
    
    return db


def create_fuels(db,df_powerplants):
    # get the fuel price property id
    price_enum = db.PropertyName2EnumId("System", "Fuel", "Fuels", "Price")
    # and the CO2 price property id
    co2_price_enum = db.PropertyName2EnumId("System", "Emission", "Emissions", "Shadow Price")
    
    # now create an emission object
    db.AddObject('CO2', EEUTILITY.Enums.ClassEnum.Emission, True)
    # get the object's system membership as always
    mem_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemEmissions,"System",'CO2')
    # and define the shadow price property for CO2
    db.AddProperty(mem_id, co2_price_enum, 1, 0,
                   None, None, None, 'prices', None, None, None, PeriodEnum.Interval)
    
    # let's det the types of fuel and associated emissions from the data
    fuel_factors =  df_powerplants[['Type','Fuel Emission Factor (g/kWh)']].dropna()
    fuel_factors = fuel_factors.drop_duplicates().set_index('Type')
    
    # now create each unique fuel
    for fuel in fuel_factors.index:
        # create the fuel object
        db.AddObject(fuel, EEUTILITY.Enums.ClassEnum.Fuel, True)
        # get it's system membership
        membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemFuels,"System",fuel)
        
        # add the fuel price property to the object
        db.AddProperty(membership_id, price_enum, 1, 0,
                       None, None, None, 'prices', None, None, None, PeriodEnum.Interval)
        
        # associate each fuel with the emissions object
        db.AddMembership(CollectionEnum.EmissionFuels,'CO2',fuel)
        
        # get the EmissionsFuels membership id
        mem_id_fuel = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.EmissionFuels,'CO2',fuel)
    
        # define the fuels co2 production rate
        co2_rate_enum = db.PropertyName2EnumId('Emission', 'Fuel', "Fuels", "Production Rate")
        db.AddProperty(mem_id_fuel, co2_rate_enum, 1, fuel_factors.loc[fuel,'Fuel Emission Factor (g/kWh)'],
                       None, None, None, None, None, None, None, PeriodEnum.Interval)
        
    return db


def create_generators(db,df_powerplants):
    # define the required property ids
    capacity_enum = db.PropertyName2EnumId("System", "Generator", "Generators", "Max Capacity")
    units_enum = db.PropertyName2EnumId("System", "Generator", "Generators", "Units")
    Heat_Rate_enum = db.PropertyName2EnumId("System", "Generator", "Generators", "Heat Rate")
    rating_enum = db.PropertyName2EnumId("System", "Generator", "Generators", "Rating")
    # for hydro:
    stor_enum = db.PropertyName2EnumId("System", "Storage", "Storages", "End Effects Method")
    stor2_enum = db.PropertyName2EnumId("System", "Storage", "Storages", "Max Volume")
    stor3_enum = db.PropertyName2EnumId("System", "Storage", "Storages", "Initial Volume")
    stor4_enum = db.PropertyName2EnumId("System", "Storage", "Storages", "Model")
    
    # which types take a rating profile instead of dispatch or hydro model? e.g. VRE
    ratings = ['PV','Offshore_Wind']

    
    for generator in df_powerplants.index:
        # add generator object
        db.AddObject(generator, EEUTILITY.Enums.ClassEnum.Generator, True)
        # get object's system membership id
        membership_id = db.GetMembershipID(EEUTILITY.Enums.CollectionEnum.SystemGenerators,"System",generator)
        
        # add the max capacity property
        db.AddProperty(membership_id, capacity_enum, 1, df_powerplants.loc[generator,'Capacity (MW)'],
                       None, None, None, None, None, None, None, PeriodEnum.Interval)
        # and the units property
        db.AddProperty(membership_id, units_enum, 1, 1,
                       None, None, None, None, None, None, None, PeriodEnum.Interval)
        
        # if it's a fuel based (dispatchable) generator:
        if df_powerplants.loc[generator,'Fuel Emission Factor (g/kWh)'] >=0:
            # add the heat rate property
            db.AddProperty(membership_id, Heat_Rate_enum, 1, df_powerplants.loc[generator,'Heat Rate'],
                           None, None, None, None, None, None, None, PeriodEnum.Interval)
            # add membership between the generator and the fuel
            db.AddMembership(CollectionEnum.GeneratorFuels,generator,df_powerplants.loc[generator,'Type'])
            
        # if the plant is VRE, add the rating property pointing to a datafile
        elif np.isin(df_powerplants.loc[generator,'Type'],ratings):
            # add the rating property
            db.AddProperty(membership_id, rating_enum, 1, 0, None, None, None,
                           'ratings', None, None, None, PeriodEnum.Interval)
            
        # finally, add the generator to the correct node
        db.AddMembership(CollectionEnum.GeneratorNodes,generator,df_powerplants.loc[generator,'Node'])
    return db
            
            
                

    
    

In [59]:
def main():
    # do all the things
    dbfile = os.path.join(os.getcwd(),'API_Example.xml')
    db = create_target_db(dbfile)
    db = connect_db(dbfile)
    db = create_datafiles(db)
    db = create_fuels(db,df_powerplants)
    
    for region in df_powerplants.Node.unique():
        db = create_region(db,region)
        db = create_node(db,region)
    db = create_lines(db,df_transmission)
    db = create_generators(db,df_powerplants)
    
    db = close_db(db)
        


In [60]:
main()

Database Saved
